# Object detection utilizing Penn-Fudan Database for Pedestrian Detection and Segmentation

Setting our environment

In [ ]:
%%shell

pip install cython
pip install -U  'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip .
unzip PennFudanPed.zip

git clone https://github.com/pytorch/vision.git
cd vision

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
import os
import numpy as np
from PIL import Image
from turtle import window_height

import transforms as T
import utils 
from engine import train_one_epoch, evaluate 

import torch
import torchvision
import torch.utils.data
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator 
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


Defining our dataset class

In [ ]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root: str, transforms=None):
        # path to folder
        self.root = root
        self.transforms = transforms 

        # load all image files, sorting them (by name) to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, 'PNGImages'))))
        self.masks = list(sorted(os.listdir(os.path.join(root, 'PedMasks'))))

    def __getitem__(self, idx):
        # get items path
        img_path = os.path.join(self.root, 'PNGImages', self.imgs[idx])
        mask_path = os.path.join(self.root, 'PedMasks', self.masks[idx])
        
        # load items as PIL Images
        img = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path)

        # convert mask (PIL Image) into numpy array
        mask = np.array(mask)
        # instances are encoded as different collors
        obj_ids = np.unique(mask)
        # ignoring first id which is the background
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
        
        # convert to tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'masks': masks,
            'image_id': image_id,
            'area': area,
            'iscrowd': iscrowd
        }

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    
    def __len__(self):
        return len(self.imgs)


Here we finetune the pre-trained model for our case, with 2 classes

In [ ]:
# load a model pre-trained on COCO (Common Objects in Context)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT')

# replace the classifier with a new one, that has
# attribute num_classes defined by user
# our case: 1 class (person) + 1 class for background
num_classes = 2
# getting the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Here we add a different backbone to the model

In [ ]:
# load a pre-trained model for classification and return
# the features
backbone = torchvision.models.mobilenet_v2(weights='DEFAULT').features

# FasterRCNN needs to know the number of output channels in a
# backbone, in this case (mobilenet_v2), 1280:
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                output_size=7, sampling_ratio=2)

model = FasterRCNN(
    backbone,
    num_classes=2,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)

Returning an instance segmentation model for our dataset

In [ ]:
def get_model_instance_segmentation(num_classes: int):
    # load an instance segmentation model pre-trained
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='DEFAULT')

    # number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features 
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # replace the pre trained head with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

Defining a help function to return our transformations

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))

    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    
    return T.Compose(transforms)

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weigths='DEFAULT')
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True,
    collate_fn=utils.collate_fn
)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

Loading the datasets

In [ ]:
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))
num_workers = 2

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=num_workers,
    collate_fn=utils.collate_fn
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=num_workers,
    collate_fn=utils.collate_fn
)

Instantiating our model and optimizer

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2
lr = 0.005
momentum = 0.9
weigth_decay = 0.0005 

model = get_model_instance_segmentation(num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weigth_decay)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
from torch.optim.lr_scheduler import StepLR

epochs = 2
for epoch in range(epochs):
  # print every 10 iterations
  train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
  # update the learning rate
  lr_scheduler.step()
  # evaluate on the test dataset
  evaluate(model, dataloader_test, device=device)

In [ ]:
img, _ = dataset_test[0]
model.eval()

with torch.no_grad():
  prediction = model([img.to(device)])

prediction

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

In [ ]:
torch.save(model.state_dict(), 'model.pth')